# Registry Persistence Mechanisms
Detects persistence registry keys

## Rule Content
```
- title: Registry Persistence Mechanisms
  id: 36803969-5421-41ec-b92f-8500f79c23b0
  description: Detects persistence registry keys
  references:
  - https://oddvar.moe/2018/04/10/persistence-using-globalflags-in-image-file-execution-options-hidden-from-autoruns-exe/
  date: 2018/04/11
  author: Karneades
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection_reg1:
      EventID: 13
      TargetObject:
      - '*\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Image File Execution Options\\*\GlobalFlag'
      - '*\SOFTWARE\Microsoft\Windows NT\CurrentVersion\SilentProcessExit\\*\ReportingMode'
      - '*\SOFTWARE\Microsoft\Windows NT\CurrentVersion\SilentProcessExit\\*\MonitorProcess'
      EventType: SetValue
    condition: selection_reg1
  tags:
  - attack.privilege_escalation
  - attack.persistence
  - attack.defense_evasion
  - attack.t1183
  - car.2013-01-002
  falsepositives:
  - unknown
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"13" AND registry_key_path.keyword:(*\\SOFTWARE\\Microsoft\\Windows\ NT\\CurrentVersion\\Image\ File\ Execution\ Options\\*\\GlobalFlag OR *\\SOFTWARE\\Microsoft\\Windows\ NT\\CurrentVersion\\SilentProcessExit\\*\\ReportingMode OR *\\SOFTWARE\\Microsoft\\Windows\ NT\\CurrentVersion\\SilentProcessExit\\*\\MonitorProcess) AND event_type:"SetValue")')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()